In [8]:
def main(dir="./papers"):
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pdfminer.high_level import extract_text
import openai
import argparse
import os
import json

# Add your API key to a file named .openai_api.txt
with open(".openai_api.txt", "r") as f:
    openai.api_key = f.read().strip()

def remove_unnecessary_parts(text):
    text = re.sub(r"(?is)\b(acknowledgments|acknowledgement|references)\b.*", "", text)
    text = re.sub(r"(?is)\bappendix\b.*", "", text)
    return text

def preprocess_text(text):
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # tokenization
    words = word_tokenize(text)

    # remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    return " ".join(words)

def reduce_length_with_gpt35(text):

    shortning_schema = {
        "type": "object",
        "properties": {
            "title": {"type": "string"},
            "author": {"type": "array", "items": {"type": "string"}},
            "journal/conference": {"type": "string"},
            "year": {"type": "string"},
            "abstract": {"type": "string"},
            "CCS_concept": {"type": "string"},
            "bib": {"type": "string"},
            "keywords": {"type": "string"},
            "introduction": {"type": "string"},
            "related_work": {"type": "string"},
            "body": {"type": "string"},
            "discussion": {"type": "string"},
            "result": {"type": "string"},
            "future_work": {"type": "string"},
            "conclusion": {"type": "string"},
            "references": {"type": "string"},
            "appendix": {"type": "string"},
            "github": {"type": "string"},
            "doi": {"type": "string"}
        },
        "required": ["title", "author", "abstract", "introduction", "related_work", "body", "discussion", "result", "future_work", "conclusion", "references", "appendix", "github", "doi", "keywords"]
    }
    """
    prompt = "Summarize this paper into 8000 tokens or less:"
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-16k",
        prompt=prompt + text
    )
    """
    prompt = "shorten this paper into 8000 tokens or less:"
    response = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content":f"{prompt}"},
            {"role": "user", "content": "Analyze the following paper and provide the required information."},
            {"role": "user", "content": f"The paper is: [{text}]"}
        ],
        functions=[{"name": "shortning_paper", "parameters": shortning_schema}],
        function_call={"name": "shortning_paper"},
        temperature=0,
    )
    # Convert the generated metadata to a JSON object
    try:
        json_result = json.loads(completion.choices[0].message.function_call.arguments)
    except:
        print("Error parsing JSON result.")
        print(completion.choices[0].message.function_call.arguments)

    return json_result

schema = {
    "type": "object",
    "properties": {
    "title": {"type": "string"},
    "author": {"type": "array", "items": {"type": "string"}},
    "jornal/conference": {"type": "string"},
    "year": {"type": "string"},
    "abstract": {"type": "string"},
    "CCS_concept": {"type": "string"},
    "bib": {"type": "string"},
    "metadata": {"type": "object"},  # ここにさらなるメタデータが入ります
    "keywords": {"type": "string"},
    "problem": {"type": "string"},
    "method": {"type": "string"},
    "interaction": {"type": "string"},
    "technical_contribution": {"type": "string"},
    "result": {"type": "string"},
    "github": {"type": "string"},
    "doi": {"type": "string"}
    },
    "required": ["title", "metadata", "keywords", "problem", "method", "interaction", "technical_contribution", "result", "github", "doi", "texts"]
}



Processing ./papers/2303.01639.pdf ...
Completed processing ./papers/2303.01639.pdf. Result saved to ./papers/2303.01639.pdf_result.json
Processing ./papers/3569474.pdf ...
Completed processing ./papers/3569474.pdf. Result saved to ./papers/3569474.pdf_result.json
Processing ./papers/3544548.3580801.pdf ...


InvalidRequestError: This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?

In [32]:
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pdfminer.high_level import extract_text
import openai
import argparse
import os
import json

# Add your API key to a file named .openai_api.txt
with open(".openai_api.txt", "r") as f:
    openai.api_key = f.read().strip()

def remove_unnecessary_parts(text):
    text = re.sub(r"(?is)\b(acknowledgments|acknowledgement|references)\b.*", "", text)
    text = re.sub(r"(?is)\bappendix\b.*", "", text)
    return text

def preprocess_text(text):
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # tokenization
    words = word_tokenize(text)

    # remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    return " ".join(words)

def get_metadata_using_gpt35_json(text):

    shortning_schema = {
        "type": "object",
        "properties": {
            "title": {"type": "string"},
            "author": {"type": "array", "items": {"type": "string"}},
            "journal/conference": {"type": "string"},
            "year": {"type": "string"},
            "abstract": {"type": "string"},
            "CCS_concept": {"type": "string"},
            "bib": {"type": "string"},
            "keywords": {"type": "string"},
            "references": {"type": "string"},
            "appendix": {"type": "string"},
            "github": {"type": "string"},
            "doi": {"type": "string"}
        },
        "required": ["title", "author", "abstract","appendix", "github", "doi", "keywords"]
    }
    prompt = f"Your task is to analyze the following academic paper and extract information according to a specified JSON format."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content":f"{prompt}"},
            {"role": "user", "content": f"The paper is: [{text}]"}
        ],
        functions=[{"name": "shortning_paper", "parameters": shortning_schema}],
        function_call={"name": "shortning_paper"},
        temperature=0,
    )

    # Convert the generated metadata to a JSON object
    try:
        json_result = json.loads(response.choices[0].message.function_call.arguments)
        return json_result
    except Exception as e:
        print(e)
        #print("Error parsing JSON result.")
        print(response.choices[0].message.function_call.arguments)
        return None
    
def reduce_length_with_gpt3(text):

    prompt = "Your task is to analyze the following academic paper and compress it within 8000 words, but over 4000 words."

    print(text)

    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-16k",
    max_tokens=6000,
    messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content":f"{prompt}"},
            {"role": "user", "content": f"The paper is: [{text}]"}
    ],
    )

    print(response)

    #return response.choices[0].text.strip()
    return response.choices[0].message.content
    

In [33]:
import os
import json
import openai

schema = {
    "type": "object",
    "properties": {
    "title": {"type": "string"},
    "author": {"type": "array", "items": {"type": "string"}},
    "jornal/conference": {"type": "string"},
    "year": {"type": "string"},
    "abstract": {"type": "string"},
    "CCS_concept": {"type": "string"},
    "bib": {"type": "string"},
    "metadata": {"type": "object"},  # ここにさらなるメタデータが入ります
    "keywords": {"type": "string"},
    "problem": {"type": "string"},
    "method": {"type": "string"},
    "interaction": {"type": "string"},
    "technical_contribution": {"type": "string"},
    "result": {"type": "string"},
    "github": {"type": "string"},
    "doi": {"type": "string"}
    },
    "required": ["title", "metadata", "keywords", "problem", "method", "interaction", "technical_contribution", "result", "github", "doi", "texts"]
}

dir="./papers"

paths = [os.path.join(dir, f) for f in os.listdir(dir) if f.endswith(".pdf")]

for pdf_filepath in paths:
    print(f"Processing {pdf_filepath} ...")

    # Extract text from the PDF file
    text = extract_text(pdf_filepath)

    # Remove unnecessary parts
    text = remove_unnecessary_parts(text)

    # Preprocess text
    text = preprocess_text(text)

    print(len(text.split()))

    # If the text is too long, reduce it with GPT-3.5
    gpt3_json = reduce_length_with_gpt3(text)


    print(gpt3_json)

    print(len(gpt3_json.split()))

    # write json
    #with open('gpt3.json', 'w') as f:
    #    json.dump(gpt3_json, f, indent=4)
    
    break



Processing ./papers/2303.01639.pdf ...
4871
WESPER Zeroshot Realtime Whisper Normal Voice Conversion Whisperbased Speech Interactions Jun Rekimoto The University Tokyo 731 Hongo Bunkyoku Tokyo Japan Sony Computer Science Laboratories Kyoto 131 Hontorocho Shimogyoku Kyotoshi Kyoto Japan rekimotoacmorg 3 2 0 2 r M 3 D S c 1 v 9 3 6 1 0 3 0 3 2 v X r Figure 1 WESPER realtime whispertonormal speech conversion mechanism consisting speechtounit STU en coder generates common speech units whispered normal utterances using selfsupervised pretraining unittospeech UTS decoder recovers speech speech units It achieves userindependent voice conversion real time ABSTRACT Recognizing whispered speech converting normal speech creates many possibilities speech interaction Because sound pressure whispered speech significantly lower normal speech used semisilent speech interaction public places without audible others Converting whispers normal speech also improves speech quality people speech hearing impa

In [ ]:
import os
import json
import openai

schema = {
    "type": "object",
    "properties": {
    "title": {"type": "string"},
    "author": {"type": "array", "items": {"type": "string"}},
    "jornal/conference": {"type": "string"},
    "year": {"type": "string"},
    "abstract": {"type": "string"},
    "CCS_concept": {"type": "string"},
    "bib": {"type": "string"},
    "metadata": {"type": "object"},  # ここにさらなるメタデータが入ります
    "keywords": {"type": "string"},
    "problem": {"type": "string"},
    "method": {"type": "string"},
    "interaction": {"type": "string"},
    "technical_contribution": {"type": "string"},
    "result": {"type": "string"},
    "github": {"type": "string"},
    "doi": {"type": "string"}
    },
    "required": ["title", "metadata", "keywords", "problem", "method", "interaction", "technical_contribution", "result", "github", "doi", "texts"]
}


def main(dir="./papers"):
    # Get all PDF files in the directory
    paths = [os.path.join(dir, f) for f in os.listdir(dir) if f.endswith(".pdf")]

    for pdf_filepath in paths:
        print(f"Processing {pdf_filepath} ...")
        
        # Extract text from the PDF file
        text = extract_text(pdf_filepath)

        # Remove unnecessary parts
        text = remove_unnecessary_parts(text)

        # Preprocess text
        text = preprocess_text(text)

        # If the text is too long, reduce it with GPT-3.5
        #if len(text.split()) > 8000:
        #    text = reduce_length_with_gpt35(text)
        
        # Generate a summary and metadata with GPT-4
        prompt = "Analyze the following paper and provide the required information. Keep the metadata in the original language. Summarize each content item in Japanese, with each item summarized to a maximum of 180 characters."
        completion = openai.ChatCompletion.create(
            #model="gpt-4-0613",
            model = "gpt-3.5-turbo-16k",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content":f"{prompt}"},
                {"role": "user", "content": "Analyze the following paper and provide the required information."},
                {"role": "user", "content": f"The paper is: [{text}]"}
            ],
            functions=[{"name": "analyze_paper", "parameters": schema}],
            function_call={"name": "analyze_paper"},
            temperature=0,
        )

        # Convert the generated metadata to a JSON object
        try:
            json_result = json.loads(completion.choices[0].message.function_call.arguments)
        except:
            print("Error parsing JSON result.")
            print(completion.choices[0].message.function_call.arguments)

        # Add the original text as a field in the JSON object
        json_result["text"] = text

        # Save the JSON object to a file
        with open(f"{pdf_filepath}_result.json", "w") as f:
            json.dump(json_result, f)

        print(f"Completed processing {pdf_filepath}. Result saved to {pdf_filepath}_result.json")

if __name__ == "__main__":
    main()


Processing ./papers/2303.01639.pdf ...
Completed processing ./papers/2303.01639.pdf. Result saved to ./papers/2303.01639.pdf_result.json
Processing ./papers/3569474.pdf ...
Completed processing ./papers/3569474.pdf. Result saved to ./papers/3569474.pdf_result.json
Processing ./papers/3544548.3580801.pdf ...
Completed processing ./papers/3544548.3580801.pdf. Result saved to ./papers/3544548.3580801.pdf_result.json
Processing ./papers/3544549.3585876.pdf ...
Completed processing ./papers/3544549.3585876.pdf. Result saved to ./papers/3544549.3585876.pdf_result.json
Processing ./papers/3581641.3584072.pdf ...
Completed processing ./papers/3581641.3584072.pdf. Result saved to ./papers/3581641.3584072.pdf_result.json
Processing ./papers/3544548.3581264.pdf ...
Completed processing ./papers/3544548.3581264.pdf. Result saved to ./papers/3544548.3581264.pdf_result.json
Processing ./papers/3491102.3517686.pdf ...
Completed processing ./papers/3491102.3517686.pdf. Result saved to ./papers/3491102